# Load Data

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("titanic").getOrCreate()
spark

In [ ]:
filepath = "/home/ubuntu/working/spark-examples/data/titanic_train.csv"
titanic_sdf = spark.read.csv(filepath, inferSchema=True, header=True)

titanic_sdf.show(5)

In [ ]:
titanic_sdf.printSchema()

In [ ]:
titanic_sdf.createOrReplaceTempView("titanic")

# EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## 1. 생존자 수 확인하기

In [ ]:
query = """
SELECT survived, count(*) as cnt
FROM titanic
GROUP BY survived
"""

sns.barplot(x='survived', y='cnt', data=spark.sql(query).toPandas())
plt.show()

## 2. Pclass 별 인원 파악하기

In [ ]:
query = """
SELECT Pclass, count(*) as cnt
FROM titanic
GROUP BY Pclass
"""

sns.barplot(x='Pclass', y='cnt', data=spark.sql(query).toPandas())
plt.show()

### Pclass 별 생존/사망자 확인

In [ ]:
query = """
SELECT Pclass, survived, count(*) as cnt
FROM titanic
GROUP BY Pclass, survived
"""

sns.barplot(x='Pclass', y='cnt', hue='survived', data=spark.sql(query).toPandas())
plt.show()

## 3. 각 Cabin 별 탑승객 확인

In [ ]:
query = """
SELECT Cabin, count(*) as cnt
FROM titanic
GROUP BY Cabin
"""
spark.sql(query).show()

뒤의 숫자 때문에 정확히 각 구역 별 몇 명인지 알 수가 없을 것 같다. 앞에 알파벳만 가져오자

In [ ]:
query = """
SELECT substr(Cabin, 0, 1) as section
FROM titanic
"""

spark.sql(query).show()

null 값이 너무 많기 때문에 가장 많이 등장한 `section`으로 `null`을 처리

In [ ]:
query = """
SELECT substr(Cabin, 0, 1) as section, survived, count(*) as cnt
FROM titanic
GROUP BY section, survived
ORDER BY section
"""

spark.sql(query).show()

C 구역의 인원이 가장 많기 때문에 Cabin 정보가 없는 사람은 임의로 `C` 구역으로 지정

In [ ]:
import pyspark.sql.functions as F

titanic_sdf = titanic_sdf.withColumn("Section", F.substring(F.col("Cabin"), 0, 1))
titanic_sdf = titanic_sdf.fillna(value="C", subset=["Section"])
titanic_sdf.show()

각 Section 별로 몇 명이 살았는지 확인하기

In [ ]:
query = """
SELECT substr(Cabin, 0, 1) as section, survived, count(*) as cnt
FROM titanic
GROUP BY section, survived
ORDER BY section
"""

sns.barplot(x='section', y='cnt', hue='survived', data=spark.sql(query).toPandas())
plt.show()

## 4. 나이대 별 탑승자 확인

In [ ]:
# udf 정의
def get_age_category(age):
    cat = ''
    
    if age <= 5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else : cat = 'Elderly'
    
    return cat


In [ ]:
spark.udf.register("get_age_category", get_age_category)

In [ ]:
# Age null 처리
import pyspark.sql.functions as F

avg_age = titanic_sdf.select(F.avg(F.col("Age")))
avg_age_row = avg_age.first()
avg_age_value = avg_age_row[0]

titanic_sdf = titanic_sdf.fillna(value=avg_age_value, subset=["Age"])
titanic_sdf.createOrReplaceTempView("titanic")

In [ ]:
query = """
SELECT get_age_category(Age) as age_cat, count(*) as cnt
FROM titanic
GROUP BY age_cat
"""

sns.barplot(x='age_cat', y='cnt', data=spark.sql(query).toPandas())
plt.show()

### 나이대 별 생존자 확인

In [ ]:
query = """
SELECT get_age_category(Age) as age_cat, survived, count(*) as cnt
FROM titanic
GROUP BY age_cat, survived
"""
plt.figure(figsize=(20, 8))
sns.barplot(x='age_cat', y='cnt', hue='survived', data=spark.sql(query).toPandas())
plt.show()

## 5. 동승자 수에 따른 생존자 확인

### 5-1. Parch, Sibsp 따로 분석하기

In [ ]:
# Parch 별 생존자
parch_query = """
SELECT Parch, survived, count(*) as cnt
FROM titanic
GROUP BY Parch, survived
"""

# Sibsp 별 생존자
sibsp_query = """
SELECT Sibsp, survived, count(*) as cnt
FROM titanic
GROUP BY Sibsp, survived
"""

plt.figure(figsize=(20, 8))

plt.subplot(121)
sns.barplot(x='Parch', y='cnt', hue='survived', data=spark.sql(parch_query).toPandas())

plt.subplot(122)
sns.barplot(x='Sibsp', y='cnt', hue='survived', data=spark.sql(sibsp_query).toPandas())

plt.show()

### 5-2 `Parch` + `Sibsp`를 합쳐 `FamilySize` 구해서 확인하기
가족 구성원 자체가 많이 없을 수록 사망자가 압도적으로 높기 때문에 두 데이터를 합쳐서 확인하기

In [ ]:
query = """
SELECT (Parch + Sibsp) as FamilySize, survived, count(*) as cnt
FROM titanic
GROUP BY FamilySize, survived
ORDER BY FamilySize
"""

plt.figure(figsize=(20, 8))

sns.barplot(x='FamilySize', y='cnt', hue='survived', data=spark.sql(query).toPandas())
plt.show()

## 6.성별 탑승자 및 사망/생존 여부 확인

In [ ]:
# 탑승자 확인.
query = """
SELECT Sex, count(*) as cnt
FROM titanic
GROUP BY Sex
"""

sns.barplot(x='Sex', y='cnt', data=spark.sql(query).toPandas())
plt.show()

남성이 더 많이 탑승했다는 것을 확인. 생존 및 사망자도 확인

In [ ]:
# 탑승자 확인.
query = """
SELECT Sex, Survived, count(*) as cnt
FROM titanic
GROUP BY Sex, Survived
"""

sns.barplot(x='Sex', y='cnt', hue='Survived', data=spark.sql(query).toPandas())
plt.show()

남성의 사망자 수가 압도적으로 높다는 것을 알 수 있음.

## 7.탑승지 별 탑승수, 사망/생존자 확인

In [ ]:
# 탑승자 확인.
query = """
SELECT Embarked, count(*) as cnt
FROM titanic
GROUP BY Embarked
"""

sns.barplot(x='Embarked', y='cnt', data=spark.sql(query).toPandas())
plt.show()

In [ ]:
# 탑승지 별 생존/사망 확인
query = """
SELECT Embarked, Survived, count(*) as cnt
FROM titanic
GROUP BY Embarked, Survived
"""

sns.barplot(x='Embarked', y='cnt', hue='Survived', data=spark.sql(query).toPandas())
plt.show()

## 8. 요금 및 나이에 따른 생존자 확인

In [ ]:
query = """
SELECT Fare, Age, Survived
FROM titanic
"""

plt.figure(figsize=(20, 8))
sns.scatterplot(x='Fare', y='Age', hue='Survived', data = spark.sql(query).toPandas())
plt.show()

500 달러 이상을 낸 Outlier가 발견되었음. 제거하고 확인(200달러 미만)

In [ ]:
query = """
SELECT Fare, Age, Survived
FROM titanic
WHERE Fare < 200
"""

plt.figure(figsize=(20, 8))
sns.scatterplot(x='Fare', y='Age', hue='Survived', data = spark.sql(query).toPandas())
plt.show()

대체적으로 요금을 많이 내고, 나이가 어릴 수록 생존자가 많음을 확인 할 수 있다.

# 머신러닝 모델 생성

## 전처리 파이프라인 생성
- 전처리 파이프라인 생성 전 null 값 여부를 먼저 확인

In [ ]:
titanic_sdf.select(
    [ F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in titanic_sdf.columns ]
).show()

Cabin은 Section으로 대체 되었고, Embarked의 null 값은 `S`로 채우기

In [ ]:
titanic_sdf = titanic_sdf.fillna("S", subset=["Embarked"])
titanic_sdf.select(
    [ F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in titanic_sdf.columns ]
).show()

In [ ]:
# 필요한 컬럼만 추출하기
query="""
SELECT Survived,
       Pclass,
       Sex,
       Age,
       (Parch + Sibsp) as FamilySize,
       Fare,
       Embarked,
       Section
FROM titanic
"""

data_df = spark.sql(query)
data_df.show()

In [ ]:
stages = []

### OneHotEncoding
- Pclass, Sex, Embarked, Section은 OneHotEncoding 처리를 수행해야 한다.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

cat_features = [
    "Pclass"
    "Sex",
    "Embarked",
    "Section"
]

for c in cat_features:
    cat_indexer = StringIndexer(inputCol=c, outputCol=c+"_idx").setHandleInvalid("keep")
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c+"_onehot"])
    stages += [cat_indexer, onehot_encoder]

stages

### StandardScaling
- FamilySize, Fare는 Standard Scaling 처리를 수행한다.

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

num_features = [
    "FamilySize",
    "Fare",
]

for n in num_features:
    num_assembler = VectorAssembler(inputCols=[n], outputCol=n+"_vector")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=n+"_scaled")
    
    stages += [num_assembler, num_scaler]

stages

In [ ]:
assemble_inputs = [c + "_onehot" for c in cat_features] + [n + "_scaled" for n in num_features]
assemble_inputs

In [ ]:
total_assembler = VectorAssembler(inputCols=assemble_inputs, outputCol='features')
stages.append(total_assembler)
stages

## 훈련 / 테스트 데이터 세트 생성

In [ ]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=42)

## 파이프라인 생성

In [ ]:
# 파이프라인 등록
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=stages)
pipeline

## 파이프라인을 이용한 데이터 변환

In [ ]:
# fit
fitted_transformer = pipeline.fit(train_df)
fitted_transformer

In [ ]:
vec_train_df = fitted_transformer.transform(train_df)
vec_train_df.printSchema()

## 모델 생성

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    maxIter=50,
    labelCol='Survived',
    featuresCol='features',
    regParam=0.1
)

lr

In [ ]:
model = lr.fit(vec_train_df)
model

# 테스트 데이터 예측

In [ ]:
# test_df 변환
vec_test_df = fitted_transformer.transform(test_df)
vec_test_df.printSchema()

In [ ]:
# vec_test_df로 예측
predictions = model.transform(vec_test_df)
predictions.select("features", "Survived", "prediction").show()

# 평가

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol = 'Survived',
    predictionCol = 'prediction',
    metricName = 'accuracy'
)

accuracy = evaluator_accuracy.evaluate(predictions)
accuracy

In [ ]:
spark.stop()